# Nosso Conjunto de Dados: Cumprimento de Sentença em Ações Coletivas vs Demais Processos

### Após a coleta inicial, foi selecionado um subconjunto de 3.000 registros para rotulagem manual, originando o DataFrame `df_rotulados`. Cada registro foi classificado com base em sua natureza processual, distinguindo entre:

- **Cumprimento de sentença em ações coletivas**, representado pelo valor **1**
- **Demais tipos de processos**, representados pelo valor **0**

Essa conversão para valores numéricos foi essencial para possibilitar análises quantitativas e treinamentos de modelos de machine learning posteriormente.

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

dfs = []
path_dados = os.getenv("PATH_DADOS")
count = 0
for arquivo in os.listdir(path_dados):
    # Só foi nescessário pegar 5 meses aleatório pois eles já teriam 10000 casos
    if arquivo.endswith('.parquet') and count<5:
      dfs.append(pd.read_parquet(f'{path_dados}/{arquivo}'))
      count=+1


SEED = 130397

df = pd.concat(dfs,ignore_index=True)
df_rotulados = pd.read_csv(f'{path_dados}/dados_rotulados.csv')

# Caminho correto dos dados rotulados
arquivo_entrada = f'{path_dados}/dados_rotulados.csv'
arquivo_saida = f'{path_dados}/dados_rotulados_filtrados.csv'

# Verifica se o arquivo existe antes de continuar
if not os.path.exists(arquivo_entrada):
    print(f"Erro: O arquivo '{arquivo_entrada}' não foi encontrado. Verifique o caminho!")
else:
    # Carregando os dados rotulados
    df_rotulados = pd.read_csv(arquivo_entrada,index_col=0).reset_index(drop=True)

    # Exibe as primeiras linhas para entender o formato
    print("Primeiras linhas do DataFrame original:")
    #print(df_rotulados.head())

    print(df_rotulados.sentiment.unique())

    # Dicionário de mapeamento para converter sentimentos em valores numéricos
    mapeamento_sentimento = {
        'Positive': 1,
        'Negative': 0,
    }

    # Verifica se a coluna 'sentiment' existe no DataFrame
    if 'sentiment' not in df_rotulados.columns:
        print("Erro: A coluna 'sentiment' não existe no DataFrame.")
    else:
        # Aplicando o mapeamento na coluna 'sentiment'
        df_rotulados['sentiment'] = df_rotulados['sentiment'].map(mapeamento_sentimento)

        # Removendo valores neutros (0) e não classificados (NaN)
        df_rotulados = df_rotulados.dropna(subset=['sentiment'])

        # Convertendo a coluna 'sentiment' para inteiro (opcional)
        df_rotulados['sentiment'] = df_rotulados['sentiment'].astype(int)

        # Salvando o novo arquivo filtrado
        df_rotulados.to_csv(arquivo_saida, index=False)

        print(f"Arquivo filtrado salvo com sucesso: {arquivo_saida}")

df_rotulados = df_rotulados.rename(columns={'sentiment':'Cumprimento_Sentenca'})
df_rotulados_positivo  = df_rotulados[df_rotulados['Cumprimento_Sentenca']==1]

Primeiras linhas do DataFrame original:
['Negative' 'Positive' 'Neutral' nan]
Arquivo filtrado salvo com sucesso: dados/dados_rotulados_filtrados.csv


## Divisão dos Dados: Estratégia Holdout

Nesta etapa, aplicamos a técnica de **Holdout** para dividir os dados em três conjuntos:

- **Treinamento (80%)**: usado para ajustar os modelos.
- **Validação (10%)**: utilizado para ajustar hiperparâmetros e evitar overfitting.
- **Teste (10%)**: reservado para a avaliação final do desempenho do modelo.

Essa divisão garante uma separação clara entre os dados usados para aprendizado e os usados para avaliação, contribuindo para a robustez e generalização dos resultados.


In [ ]:
from sklearn.model_selection import train_test_split

def split_holdout(df):
    """
    Separa um dataframe em conjuntos de treino (80%), validação (10%) e teste (10%).

    Parâmetros:
    df (DataFrame): O dataframe contendo os dados não rotulados.

    Retorna:
    tuple: (treino, validacao, teste)
    """

    # Separar 80% para treino e 20% para validação + teste
    treino, temp = train_test_split(df, test_size=0.2, random_state=42)

    # Separar 10% para validação e 10% para teste
    validacao, teste = train_test_split(temp, test_size=0.5, random_state=42)

    return treino, validacao, teste

## Extração de Características com BERT

Nesta etapa, utilizamos o modelo **ModernBERT-base** da Hugging Face para transformar os textos jurídicos em **vetores numéricos de características** (embeddings), que serão usados como entrada para modelos de machine learning.

O processo é realizado da seguinte forma:

- Cada texto é tokenizado e processado por um modelo pré-treinado da arquitetura BERT.
- A saída da última camada oculta é utilizada para gerar um vetor de características representando semanticamente o texto.
- O vetor resultante é obtido pela média dos embeddings de cada token, garantindo uma representação compacta e significativa.
- As características extraídas são combinadas com seus respectivos rótulos (`Cumprimento_Sentenca`) para posterior treinamento e avaliação.

Essa etapa é aplicada a todos os conjuntos de dados: **treinamento**, **validação** e **teste**, tanto para os exemplos rotulados quanto para os não rotulados.

Além disso, os resultados são armazenados em arquivos `.parquet`, evitando a reprocessamento desnecessário e otimizando o tempo de execução nos ciclos futuros de experimentação.


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

def extrair_caracteristicas_de_dataframe(
    df: pd.DataFrame, tokenizer: AutoTokenizer, modelo: AutoModel
) -> pd.DataFrame:
    """Extrai características de um DataFrame contendo textos e rótulos.

    Args:
        df (pd.DataFrame): DataFrame contendo as colunas 'texto' e 'Cumprimento_Sentença'.
        tokenizer (AutoTokenizer): O tokenizer do Hugging Face.
        modelo (AutoModel): O modelo pré-treinado do Hugging Face.

    Returns:
        pd.DataFrame: DataFrame com colunas ['rotulo', 'texto', 'caracteristicas']
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(device)
    caracteristicas = []

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        texto = row["texto"]
        rotulo = row.get("Cumprimento_Sentenca", 0)  # Assume 0 se não houver valor

        if pd.isna(texto):
            continue  # Ignorar linhas sem texto

        # Tokenizar o texto
        entradas = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
        entradas = {k: v.to(device) for k, v in entradas.items()}  # Mover inputs para a GPU

        # Passar o texto tokenizado pelo modelo para obter as saídas
        with torch.no_grad():
            saidas = modelo(**entradas)

        # Extrair os embeddings da última camada do modelo
        ultimos_estados_ocultos = saidas.last_hidden_state

        # Calcular a média dos estados ocultos para obter um vetor de características
        vetor_de_caracteristicas = (
            ultimos_estados_ocultos.mean(axis=1).squeeze().detach().cpu().numpy()
        )

        # Adicionar à lista de características
        caracteristicas.append((rotulo, texto, vetor_de_caracteristicas))

    return pd.DataFrame(caracteristicas, columns=["rotulo", "texto", "caracteristicas"])


# Amostragem e divisão dos DataFrames
tamanho = 10000
df_nao_rotulado = df.sample(tamanho - len(df_rotulados_positivo))
cumprimento_treino, cumprimento_validacao, cumprimento_teste = split_holdout(df_rotulados_positivo)
nao_rotulado_treino, nao_rotulado_validacao, nao_rotulado_teste = split_holdout(df_nao_rotulado)

# Carregar o modelo e o tokenizer do Hugging Face para o processamento de texto
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
modelo = AutoModel.from_pretrained("answerdotai/ModernBERT-base")

if 'caracteristicas_treino' in os.listdir():
  caracteristicas_treino = pd.read_parquet('caracterisiticas_treino.parquet')
else:
  caracteristicas_treino = pd.concat([
      extrair_caracteristicas_de_dataframe(cumprimento_treino, tokenizer, modelo),
      extrair_caracteristicas_de_dataframe(nao_rotulado_treino, tokenizer, modelo)
  ])
if 'caracteristicas_validacao.parquet' in os.listdir():
  caracteristicas_validacao = pd.read_parquet('caracterisiticas_validacao.parquet')
else:
  caracteristicas_validacao = pd.concat([
    extrair_caracteristicas_de_dataframe(cumprimento_validacao, tokenizer, modelo),
    extrair_caracteristicas_de_dataframe(df_rotulados[df_rotulados['Cumprimento_Sentenca']==0],tokenizer, modelo),
])
if 'caracteristicas_teste.parquet' in os.listdir():
  caracteristicas_teste = pd.read_parquet('caracterisiticas_teste.parquet')
else:
  caracteristicas_teste = pd.concat([
      extrair_caracteristicas_de_dataframe(cumprimento_teste, tokenizer, modelo),
      extrair_caracteristicas_de_dataframe(nao_rotulado_teste, tokenizer, modelo)
  ])


##Salvamento dos Dados Vetorizados

Após a extração das características com o modelo BERT, os conjuntos de **treinamento**, **validação** e **teste** foram reorganizados com `reset_index` para garantir consistência nos índices e, em seguida, salvos em formato `.parquet`.

Esse formato foi escolhido por sua eficiência em leitura e escrita de grandes volumes de dados, facilitando o reuso dos vetores sem a necessidade de reprocessamento futuro.

In [ ]:
caracterisiticas_treino = caracteristicas_treino.reset_index(drop=True)
caracterisiticas_validacao = caracteristicas_validacao.reset_index(drop=True)
caracterisiticas_teste = caracteristicas_teste.reset_index(drop=True)

caracteristicas_treino.to_parquet('caracterisiticas_treino.parquet')
caracteristicas_validacao.to_parquet('caracterisiticas_validacao.parquet')
caracteristicas_teste.to_parquet('caracterisiticas_teste.parquet')


## Visualização das Características com PCA (3D)

Para compreender melhor a separabilidade entre as classes com base nos vetores gerados pelo modelo BERT, aplicamos **Análise de Componentes Principais (PCA)** com 3 componentes.

### O que foi feito:

- A coluna de características vetorizadas foi convertida para um array NumPy.
- Aplicamos **PCA** para reduzir a dimensionalidade dos vetores de alta dimensão para apenas **3 componentes principais**, preservando o máximo possível da variância original.
- Geramos um **gráfico de dispersão 3D** com os vetores projetados, colorindo os pontos de acordo com seus rótulos:
  - `Rótulo 1` (Cumprimento de Sentença de Ação Coletiva)
  - `Rótulo 0` (Demais Processos)

### Objetivo:

Essa visualização permite uma inspeção visual da separabilidade entre as classes no espaço vetorial, podendo indicar se os embeddings possuem potencial discriminativo para tarefas de classificação.

A transparência (`alpha=0.02`) foi aplicada aos pontos do rótulo 0 para facilitar a visualização da densidade de pontos positivos.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Carregar os dados
caracteristicas_treino = pd.read_parquet('caracterisiticas_treino.parquet')

# Converter a coluna 'caracteristicas' para um array NumPy
caracteristicas = np.array(caracteristicas_treino['caracteristicas'].tolist())

# Aplicar PCA para reduzir a dimensionalidade para 3 componentes
pca = PCA(n_components=3)
caracteristicas_pca = pca.fit_transform(caracteristicas)

# Imprimir a variância explicada
print("Variância explicada por cada componente:", pca.explained_variance_ratio_)
print("Variância total explicada:", np.sum(pca.explained_variance_ratio_))

# Criar o gráfico 3D de dispersão
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(caracteristicas_pca[caracteristicas_treino['rotulo'] == 1, 0],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 1, 1],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 1, 2],
           label='Rótulo 1', marker='x')
ax.scatter(caracteristicas_pca[caracteristicas_treino['rotulo'] == 0, 0],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 0, 1],
           caracteristicas_pca[caracteristicas_treino['rotulo'] == 0, 2],
           label='Rótulo 0', marker='o',alpha=0.02)

ax.set_xlabel('Componente Principal 1')
ax.set_ylabel('Componente Principal 2')
ax.set_zlabel('Componente Principal 3')
ax.set_title('PCA das Características em 3D')
ax.legend()
plt.show()


## Formatação e Análise dos Rótulos

Nesta etapa, os rótulos do conjunto de treino foram convertidos para o formato esperado por modelos de aprendizado semi-supervisionado:

- Rótulo **1**: representa um exemplo **positivo** (cumprimento de sentença).
- Rótulo **-1**: representa um exemplo **não rotulado** (demais processos, tratados como negativos ou desconhecidos).

Como a função `np.bincount()` não aceita valores negativos, foi aplicada uma transformação temporária, somando **1** a cada rótulo, para deslocar os valores ao intervalo não-negativo:

- O valor `-1` se torna `0` (primeira posição do array de contagens).
- O valor `1` se torna `2` (terceira posição).

Esse processo permitiu contar de forma correta e eficiente quantos exemplos rotulados e não rotulados estavam presentes no conjunto de treino.

In [ ]:
X_train = caracteristicas_treino['caracteristicas']
y_train = caracteristicas_treino['rotulo'].tolist()
X_valid = caracteristicas_validacao['caracteristicas']
y_valid = caracteristicas_validacao['rotulo'].tolist()


# Converter os rótulos combinados para um novo formato
# Se o rótulo for 0 (não rotulado), converter para -1
# Se o rótulo for 1 (positivo), manter como 1
y_train_formatted = np.array([-1 if x == 0 else 1 for x in y_train])

# Contar as ocorrências de cada rótulo (-1 e 1) nos rótulos formatados
# np.bincount conta o número de ocorrências de cada valor em um array
# Como o np.bincount espera apenas inteiros não negativos, ele não funciona diretamente com o valor -1
# Para contornar isso, podemos somar 1 a cada elemento antes de contar
# Isso desloca os valores para o intervalo de inteiros não negativos
counts = np.bincount(y_train_formatted + 1)

# Imprimir as contagens de -1 e 1
# counts[0] corresponde à contagem de -1 (originalmente 0 no intervalo deslocado)
# counts[2] corresponde à contagem de 1 (originalmente 2 no intervalo deslocado)
print(f"Count of -1 (unlabeled): {counts[0]}")
print(f"Count of 1 (positive): {counts[2]}")

In [ ]:
X_train = np.array(X_train.tolist())
X_valid = np.array(X_valid.tolist())

y_train = np.array(y_train)
y_valid = np.array(y_valid)

## Aprendizado Positivo-Não Rotulado com Elkan e Noto (PU Learning)

Nesta etapa, aplicamos a técnica de **PU Learning (Positive-Unlabeled Learning)** utilizando a abordagem proposta por **Elkan e Noto (2008)** para treinar um classificador a partir de um conjunto de dados contendo apenas exemplos **positivos rotulados** e **exemplos não rotulados** (que podem ser positivos ou negativos).

A implementação foi feita com a biblioteca `pulearn`, que oferece suporte nativo à técnica, estendendo os classificadores do `scikit-learn`.

### Abordagem de Elkan e Noto: Etapas Principais

1. **Treinamento de um Classificador Inicial**  
   Treinamos um modelo probabilístico (neste caso, um `MLPClassifier`) para estimar a probabilidade de uma amostra ter sido **rotulada**.  

2. **Estimativa de Probabilidade de Rotulagem**  
   Após o treinamento, reservamos uma fração dos exemplos positivos para estimar a probabilidade \\( P(s = 1 \mid y = 1) \\), ou seja, a probabilidade de um exemplo verdadeiramente positivo ter sido rotulado.

3. **Ajuste das Probabilidades Estimadas**  
   Para cada exemplo **não rotulado**, o modelo estima a probabilidade de ele ter sido rotulado.  
   Essa probabilidade é então **corrigida** dividindo-a por \\( P(s = 1 \mid y = 1) \\), permitindo estimar com mais precisão se a amostra **é de fato positiva**.

### Implementação com `pulearn`

Utilizamos o `ElkanotoPuClassifier` da biblioteca `pulearn`, com os seguintes parâmetros:

- **Estimador base**: `MLPClassifier`, um classificador neural do `sklearn`.
- **`hold_out_ratio=0.10`**: 10% das amostras positivas foram reservadas para estimar a probabilidade de rotulagem.
- **Rótulos de entrada**:
  - `1` para exemplos **positivos rotulados**.
  - `-1` para exemplos **não rotulados**.

### Vantagens da Abordagem
- Corrige a **rotulagem incompleta** presente nos dados PU.
- Permite treinar modelos mesmo **sem exemplos negativos rotulados**.
- Ideal para cenários onde é difícil obter exemplos negativos confiáveis.


In [ ]:
from pulearn import ElkanotoPuClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, matthews_corrcoef
import helpers.classification

# Inicializar o RandomForestClassifier
# n_jobs=-1: Utiliza todos os núcleos de CPU disponíveis para processamento paralelo
# random_state=271828: Semente para o gerador de números aleatórios, garantindo reprodutibilidade
model = MLPClassifier(random_state=271828)

# Inicializar o ElkanotoPuClassifier com o RandomForestClassifier como estimador base
# hold_out_ratio=0.30: Proporção de amostras positivas reservadas para estimar P(s=1|y=1)
pu_estimator = ElkanotoPuClassifier(estimator=model, hold_out_ratio=0.10)

# Treinar o classificador PU no conjunto de dados combinado
# X_train: Matriz de características contendo amostras positivas e não rotuladas
# y_train_formatted: Rótulos formatados como -1 para não rotuladas e 1 para amostras positivas
pu_estimator.fit(X_train, y_train_formatted)

# Prever os rótulos para o conjunto de validação usando o classificador PU treinado
y_valid_pred = pu_estimator.predict(X_valid)

# Exibir as métricas de classificação
helpers.classification.print_classification_metrics(y_valid, y_valid_pred)